# Week 04 Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm19prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.

Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


In [3]:
import numpy as np
import pandas as pd
import csv

In [64]:
# Load data
header_list = ['STN', 'YYYYMMDD', 'TG', 'TN', 'TX', 'SQ', 'DR', 'RH']
df = pd.read_csv('C:/Data_Science_for_Life_Sciences_MASTER/programming1/BFVM19PROG1/data/KNMI_20181231.txt.tsv', comment='#', names = header_list)
df.head()

c:\python\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,STN,YYYYMMDD,TG,TN,TX,SQ,DR,RH
0,209,20010130,,,,,,NaN
1,209,20010131,,,,,,NaN
2,209,20010201,,,,,,NaN
3,209,20010202,,,,,,NaN
4,209,20010203,,,,,,NaN


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [65]:
#replace cells with spaces to NaN
df.TG = pd.to_numeric(df.TG, errors='coerce')
df.TN = pd.to_numeric(df.TN, errors='coerce')
df.TX = pd.to_numeric(df.TX, errors='coerce')
df.SQ = pd.to_numeric(df.SQ, errors='coerce')
df.DR = pd.to_numeric(df.DR, errors='coerce')
df.RH = pd.to_numeric(df.RH, errors='coerce')
df.tail()

,STN,YYYYMMDD,TG,TN,TX,SQ,DR,RH
331311,391,20181227,12.0,-18.0,47.0,28.0,0.0,0.0
331312,391,20181228,7.0,-29.0,30.0,23.0,0.0,0.0
331313,391,20181229,59.0,25.0,92.0,0.0,15.0,5.0
331314,391,20181230,78.0,52.0,87.0,3.0,42.0,17.0
331315,391,20181231,89.0,74.0,97.0,0.0,0.0,-1.0


In [66]:
#change data formats
df.STN = df.STN.astype('int64')
df.YYYYMMDD = pd.to_datetime(df['YYYYMMDD'].astype(str), format='%Y%m%d')
df.tail()
# select only required  columns and rename
df = df[['STN', 'YYYYMMDD', 'TG', 'TN', 'TX']]
df.rename(columns = {'STN': 'station', 'YYYYMMDD': 'Date', 'TG': 'Tmean', 'TN': 'Tmin', 'TX': 'Tmax'}, inplace = True)
df.tail()

,station,Date,Tmean,Tmin,Tmax
331311,391,2018-12-27,12.0,-18.0,47.0
331312,391,2018-12-28,7.0,-29.0,30.0
331313,391,2018-12-29,59.0,25.0,92.0
331314,391,2018-12-30,78.0,52.0,87.0
331315,391,2018-12-31,89.0,74.0,97.0


In [67]:
# select one weather station
df = df[df.station == 270]

In [68]:
#change temperatures to celcius degrees
dfcp = df[:]
dfcp[['Tmean', 'Tmin', 'Tmax']] = dfcp[['Tmean', 'Tmin', 'Tmax']]/10

In [69]:
# remove the leap year
dfcp['year'] = pd.DatetimeIndex(dfcp['Date']).year
dfcp['month'] = pd.DatetimeIndex(df['Date']).month
dfcp['day'] = pd.DatetimeIndex(df['Date']).day
#weather = dfcp.loc[~(dfcp['day']==29) & ~(dfcp['month']==2)]
#weather.head()
leap_day = (dfcp.day == 29) & (dfcp.month == 2)

weather = dfcp[leap_day == False]

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [70]:
#Test your outcome
weather.info()
#write code to check if you have done the above

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6935 entries, 97641 to 104580
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  6935 non-null   int64         
 1   Date     6935 non-null   datetime64[ns]
 2   Tmean    6935 non-null   float64       
 3   Tmin     6935 non-null   float64       
 4   Tmax     6935 non-null   float64       
 5   year     6935 non-null   int64         
 6   month    6935 non-null   int64         
 7   day      6935 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(4)
memory usage: 487.6 KB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [71]:
# check which year is the latest year of weather measurements
weather.year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [105]:
# split the data by year into two data frames
# all measurements in the years before 2018
weather_bf_18 = weather[weather.year < 2018]

# all measurements for the year 2018
weather_18 = weather[weather.year == 2018]
weather_18

,station,Date,Tmean,Tmin,Tmax,year,month,day
104216,270,2018-01-01,6.0,4.0,7.9,2018,1,1
104217,270,2018-01-02,5.6,3.1,7.5,2018,1,2
104218,270,2018-01-03,7.5,5.3,9.2,2018,1,3
104219,270,2018-01-04,7.3,5.8,9.1,2018,1,4
104220,270,2018-01-05,6.0,4.0,7.6,2018,1,5
...,...,...,...,...,...,...,...,...
104576,270,2018-12-27,5.7,5.3,6.2,2018,12,27
104577,270,2018-12-28,7.1,5.8,8.1,2018,12,28
104578,270,2018-12-29,8.5,6.9,10.2,2018,12,29
104579,270,2018-12-30,8.0,6.8,9.0,2018,12,30


In [73]:
# Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. 
# So we look for the minimum value out of all january-01 minimum values (regardless the year). 
#Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day.

# group the dataset with the measurements before 2018 by day
weather_uni = weather.groupby(['month', 'day']).agg({'Tmin': [min], 'Tmax': ['max']}).reset_index()
weather_uni.columns = ['month', 'day', 'Tmin','Tmax']

In [74]:
#def month_day(df_multipleyears):
#    #your code to reform data here
#    
#    print(df_groupedbymonthday)

In [118]:
#Test your code
#def test_reformed(df)
    #
#    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
#    month_day(df)

#test_reformed(weather_bf_18)

SyntaxError: invalid syntax (Temp/ipykernel_9348/3390980078.py, line 2)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [ ]:
#your code here
#your motivation here
# smoothing shall reduce the noise and reveal long term trends

In [190]:
weather_18 = weather_18.copy()
weather_18
weather_uni = weather_uni.copy()
weather_uni

# start with a moving average as a basis (see chunk below the plot for motivation )
rolling_windows = weather_18.Tmean.rolling(10, min_periods=1)
weather_18['mov_avg'] = rolling_windows.mean()

from scipy.ndimage import gaussian_filter
Gaussian = gaussian_filter(weather_18.Tmean, sigma=2)

# simple exponential smoothing (might not be the best because of seasonality)
from statsmodels.tsa.api import SimpleExpSmoothing

index= weather_18.Date
data = pd.Series(weather_18.Tmean.values, index)

SES = SimpleExpSmoothing(data, initialization_method="estimated").fit(smoothing_level=0.2, optimized=False)
fcastSES = SES.forecast(3).rename(r"$\alpha=%s$" % fit3.model.params["smoothing_level"])
SES.fittedvalues

c:\python\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


Date
2018-01-01    7.520000
2018-01-02    7.216000
2018-01-03    6.892800
2018-01-04    7.014240
2018-01-05    7.071392
                ...   
2018-12-27    6.035542
2018-12-28    5.968434
2018-12-29    6.194747
2018-12-30    6.655798
2018-12-31    6.924638
Length: 365, dtype: float64

In [77]:
from bokeh.plotting import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.models import Band
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [189]:
# define figure 
fig = figure(plot_width = 750,
            plot_height = 700,
            tools = '',
            toolbar_location = None)
# plot original line
fig.line(weather_18.Date, weather_18.Tmean, legend_label = "average temperature")
fig.line(weather_18.Date, weather_18.mov_avg, line_color = 'red', legend_label = 'moving average')
fig.line(weather_18.Date, SES.fittedvalues, line_color = 'green', legend_label = 'SES')
fig.line(weather_18.Date, Gaussian, line_color = 'orange', legend_label = 'Gaussian')
# plot trenlines with min and max values
#fig.line(weather_18.Date, weather_uni.Tmin.values, line_color = 'grey', line_dash = 'dotdash')
#fig.line(weather_18.Date, weather_uni.Tmax.values, line_color = 'grey', line_dash = 'dotdash')
show(fig)    

The moving average was proposed by this tutorial (https://medium.com/@teamastertoast/smoothing-time-series-in-python-a-walkthrough-with-covid-19-data-b4ccfb980a61), and seemed reasonable as a starting point, however it introduces a shifting of the signal to the right.

Next method is gaussian filter, which is frequently applied in image analysis, here the peaks are reduced. As higher the sigma value as smoother the curve and as smaller the peaks. I applied it following the linear smoothers mentioned in the article provided by Fenna.

Third method is the simple exponential smoothing, which is according to this benchmarking study (https://core.ac.uk/download/pdf/188772251.pdf) the best smoothing method when forecasting temperature data. Surprisingly it outperformed the Holt-Winter method, which is assumed to be the best.

I made my decision based on this publication because i thought, if the smoothing is performed as a base for forecasting it needs to be reliable, reducing the noise and visualizes trends better.

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

In [203]:
weather_prep = pd.DataFrame(dict(Date = weather_18.Date,
                       lower = weather_uni.Tmin.values,
                       upper = weather_uni.Tmax.values,
                        Tmean = weather_18.Tmean))

weather_prep
source = ColumnDataSource(weather_prep)

index= weather_18.Date
data_18 = pd.Series(weather_18.Tmean.values, index)

SES_18 = SimpleExpSmoothing(data_18, initialization_method="estimated").fit(smoothing_level=0.2, optimized=False)
fcastSES_18 = SES_18.forecast(3).rename(r"$\alpha=%s$" % fit3.model.params["smoothing_level"])
SES_18.fittedvalues

data_lower = pd.Series(weather_uni.Tmin.values, index)

SES_lower = SimpleExpSmoothing(data_lower, initialization_method="estimated").fit(smoothing_level=0.2, optimized=False)
fcastSES_lower = SES_lower.forecast(3).rename(r"$\alpha=%s$" % fit3.model.params["smoothing_level"])
SES_lower.fittedvalues

data_upper = pd.Series(weather_uni.Tmax.values, index)

SES_upper = SimpleExpSmoothing(data_upper, initialization_method="estimated").fit(smoothing_level=0.2, optimized=False)
fcastSES_upper = SES_upper.forecast(3).rename(r"$\alpha=%s$" % fit3.model.params["smoothing_level"])
SES_upper.fittedvalues

weather_SES = pd.DataFrame(dict(lower = SES_lower.fittedvalues,
                               upper = SES_upper.fittedvalues,
                               mean = SES_18.fittedvalues))

source_SES = ColumnDataSource(weather_SES)

# define plot
p = figure(title = 'Average temperature in the region Groningen', 
           x_axis_label='Days', y_axis_label='Temperature [°C]',
           x_axis_type = 'datetime',
           #y_range = (weather_SES.lower.min(), weather_SES.upper.max())
           y_range = (-15, 35), plot_height = 400)
band = Band(base='Date', lower='lower', upper='upper', source=source_SES, fill_color = 'red', fill_alpha = 0.2, line_color = 'red')
p.add_layout(band)
p.line(x = 'Date', y = 'mean',source = source_SES, legend_label = "average temperature", color = 'black')

show(p)

c:\python\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
c:\python\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
c:\python\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather